### Developing the LSH method from scratch.

In [67]:
from math import sqrt

def cosine_distance(x, y):
    return sum([i*j for i,j in zip(x, y)])/(sqrt(sum([i*i for i in x]))*sqrt(sum([i*i for i in y])))

class Image:
    def __init__(self, f, d):
        self.projections = np.random.randn(f, d)
        
    def hash_point(self, x):
        bools = (np.dot(x, self.projections.T) > 0).astype('int')
        return ''.join(bools.astype('str'))

class LSHIndex:
    def __init__(self, f, b, d):
        self.hash_tables = []
        self.b = b
        self.X = {}
        self.image = Image(f, d)
        for i in range(self.b):
            self.hash_tables.append({})
    
    def index(self, key, x):
        # storing all input vectors + its key in a dictionary to use it later
        self.X[key] = x
        # hash the input vector into f dimentions
        generated_vector = self.image.hash_point(x)
        # calculating how many charachters each b segment has
        part_len = len(generated_vector)//self.b

        for i in range(self.b):
            if (i != (self.b - 1)):
                binary = generated_vector[i*part_len: (i+1)*part_len]
            else:
                binary = generated_vector[i*part_len:]

            # binary to decimal
            decimal = int(binary, 2)
            # store each decimal representation corresponding its key in the i'th hash table
            self.hash_tables[i][key] = decimal

        return self.hash_tables

    def query(self, q):
        # for storing candidate vectors keys, define a list 
        candid_vectors_keys = []
        # for storing candidate vectors cosine similarity with q, define a list
        all_cosines = []
        # we again hash q with same method as bedore
        generated_vector = self.image.hash_point(q)
        part_len = len(generated_vector)//self.b

        for i in range(self.b):
            if (i != (self.b - 1)):
                binary = generated_vector[i*part_len: (i+1)*part_len]
            else:
                binary = generated_vector[i*part_len:]

            decimal = int(binary, 2)

            # we find which key(s) have the same decimal representation of hashed value and if there exist such a key, we store that key
            candid_vector_key = next((key for key, decimals in self.hash_tables[i].items() if decimals == decimal), None)
            if candid_vector_key != None:
              if candid_vector_key not in candid_vectors_keys:
                candid_vectors_keys.append(candid_vector_key)
            
        # getting original vectors for the corresponding key values
        for key in candid_vectors_keys:
            all_cosines.append((key, cosine_distance(self.X.get(key), q)))

        all_cosines.sort(key=lambda i:i[1], reverse=True)

        results = []
        for i in range(10):
          if all_cosines[i][1] < 1: # excluding the q vector similarity with its self
            results.append(all_cosines[i][0])
        
        return results

### Here we imported libraries + imported a pre-trained gensim model named glove-wiki-gigaword-300 (we use 300 model since we want to vectorize every word into a 300 dimention vector !). In addition, we imported the csv dataset.

In [38]:
import gensim.downloader as api
import gensim
import csv
import numpy as np

model = api.load('glove-wiki-gigaword-300')

# Mounting dataset from Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

with open('/content/gdrive/MyDrive/title.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### for each sentences in the csv dataset, given that we have the vectorized representation of words using pre-trained model, we calculated avg of words' vectors.

In [59]:
sentence_vectors = []
for sentence in data:
  vector = [0]*300
  word_number = 0
  for word in sentence[1].split():
    try:
      word_number += 1
      vector = np.add(model[word], vector)
    except:
      continue
  
  sentence_vectors.append(np.divide(vector, word_number))

In [74]:
target_sentence_1 = 'end of world'
vector = [0]*300
for word in target_sentence_1.split():
  vector = np.add(model[word], vector)

target_sentence_1_vector = np.divide(vector, 3)

target_sentence_2 = 'he and his friends'
vector = [0]*300
for word in target_sentence_2.split():
  vector = np.add(model[word], vector)

target_sentence_2_vector = np.divide(vector, 3)

### We first indexed every senteces into hash tables using LSH.index, then we made query (our query here is the target senteces). Here we printed out IDs of sentences that are similar to the target sentences.

In [75]:
N = len(sentence_vectors)
d = 300
f = 100
b = 20
r = 5
k = 10

LSH = LSHIndex(f, b, d)
for i in range(N):
  LSH.index(i, sentence_vectors[i])

print('similar text to "end of world" by ID : ', LSH.query(target_sentence_1_vector))
print('similar text to "he and his friends" by ID : ', LSH.query(target_sentence_2_vector))

similar text to "end of world" by ID :  [79, 132, 445, 16, 86, 143, 9978, 19, 12]
similar text to "he and his friends" by ID :  [285, 79, 253, 744, 16, 39, 93, 143, 24, 12]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.
